# 期末專案：LLM Information Extraction(Sentiment classification)
**Author:** 陳文薇<br>

# 連接雲端資料夾

In [1]:
import os

from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/sma/proj') #切換該目錄
os.listdir() #確認目錄內容

Mounted at /content/drive


['stanford-corenlp.zip',
 'stanza_corenlp',
 'stanford-corenlp-latest.zip',
 'stanford-corenlp',
 'corenlp_server-c7683579fc3048c8.props',
 'corenlp_server-8421c828ca0a4b27.props',
 'corenlp_server-33f5eb266f804605.props',
 'corenlp_server-5a0c78ec523c4fbf.props']

# 套件安裝

In [2]:
!pip install jieba

!pip install sentence-transformers
!pip install transformers
!pip install sentencepiece

# 用於 transformer 加速
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

!pip install langchain-chroma
!pip install pypdf
!pip install langchain
!pip install huggingface_hub
!pip install grandalf
!pip install pandas
!pip install ctransformers
!pip install langchain_community
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
!pip install langchain_core
!pip install nltk

In [7]:
import os
from getpass import getpass

import pandas as pd
import ast
import json
import re
import jieba
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import BitsAndBytesConfig # huggingface 量化

from langchain_community.llms import HuggingFaceEndpoint

from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [8]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# HuggingFace Token

輸入 HuggingFace Token
- get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

In [9]:
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

··········


### 載入 LLM 模型

#### HuggingFacePipeline


In [10]:
quant_config = BitsAndBytesConfig(load_in_4bit=True)

print(quant_config.is_quantizable())
print(quant_config.quantization_method())

True
fp4


In [11]:
model_id = "google/gemma-1.1-7b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=HUGGINGFACEHUB_API_TOKEN)
device_map="auto" #使用 GPU 加速
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quant_config, token=HUGGINGFACEHUB_API_TOKEN)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


#### HuggingFaceEndpoint
- Langchian 整合 huggingface severless api (https://huggingface.co/docs/api-inference/index)




In [12]:
# repo_id: HuggingFace 上的模型名稱，本範例使用 gemma 7b instruct 版本
    # huggingface: https://huggingface.co/google/gemma-1.1-7b-it
# temperature: 模型回答的活潑程度，0~1 越大每次回答的多樣性越高

llm = HuggingFaceEndpoint(
    repo_id="google/gemma-1.1-7b-it", temperature=1.0,model_kwargs={'token':HUGGINGFACEHUB_API_TOKEN, 'use_cache':False}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# 資料集實作 Information Extraction (Tagging)
- https://python.langchain.com/docs/use_cases/tagging/

此部分為情緒分析

In [13]:
maildata = pd.read_csv("../raw_data/Phishing_Email.csv")

#去除文章內容為空值的筆數
maildata.dropna(subset = ['Email Text'], axis=0, how='any', inplace=True)

#新增['sentence']欄位，用'。'取代'\n\n'，並移除'\n'
maildata['sentence'] = maildata['Email Text'].str.replace(r'\n\n','。', regex=True)
maildata['sentence'] = maildata['sentence'].str.replace(r'\n','', regex=True)
# 刪除標點符號/數字/換行符號
maildata["sentence"] = maildata["sentence"].apply(lambda x: re.sub(r'[^\w\s]','', str(x)))  #只留下英文字母和空格(包含換行符號)
maildata['sentence'] = maildata["sentence"].str.replace(r'[\d]+', '', regex=True).astype(str) #去除數字
maildata["sentence"] = maildata["sentence"].apply(lambda x: re.sub(r'[\n_-]+',' ', x)) #將換行符號替換成空格


# 去除空值
# dataset 有些郵件是 empty
# 經過 nltk 套件會有些東西變 nan
maildata = maildata[maildata["sentence"] != "empty"]
maildata = maildata[maildata["sentence"] != "nan"]

#移除內文中的網址
maildata['sentence'] = maildata['sentence'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
maildata['sentence'] = maildata['sentence'].str.split("[,，。！!？?]{1,}")

maildata = maildata.explode('sentence').reset_index(drop=True)
# 把長度小於1的sentence刪掉
maildata = maildata[maildata["sentence"].str.len() > 1]
maildata = maildata.sample(n=100, random_state=222)
# 將不必要的欄位 以及 nan 刪掉
maildata = maildata.rename(columns={"Unnamed: 0":"index"})
maildata.head(5)

,index,Email Text,Email Type,sentence
6974,7192,\nNow you can have HUNDREDS of lenders compete...,Phishing Email,Now you can have HUNDREDS of lenders compete f...
15453,15914,PUBLIC ANNOUNCEMENT:The new .NAME domain exten...,Phishing Email,PUBLIC ANNOUNCEMENTThe new NAME domain extensi...
3275,3386,get the best price on your next car ! exclusiv...,Phishing Email,get the best price on your next car exclusive...
4186,4333,"On Thu, 1 Aug 2002 17:10:48 +0100, John Hinsle...",Safe Email,On Thu Aug John Hinsley wrote No the prob...
6008,6198,santa barbara nexus ezine - great article pert...,Phishing Email,santa barbara nexus ezine great article perta...


In [14]:
class Email(BaseModel):
    情緒: str = Field(..., description="判斷整封郵件總體的情緒。", enum=["正向", "中性", "負向"])
    信件種類: str = Field(..., description="判斷信件的種類，例如釣魚信或安全信。", enum=["釣魚信", "安全信", "其他"])
    關鍵字: str = Field(..., description="信件中提及的關鍵字，並以逗號','分隔。")

email_parser = JsonOutputParser(pydantic_object=Email)

# 透過 get_format_instructions() 可以看到如何告訴 LLM 生成我們想要的格式的一些指令
email_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"\\u60c5\\u7dd2": {"title": "\\u60c5\\u7dd2", "description": "\\u5224\\u65b7\\u6574\\u5c01\\u90f5\\u4ef6\\u7e3d\\u9ad4\\u7684\\u60c5\\u7dd2\\u3002", "enum": ["\\u6b63\\u5411", "\\u4e2d\\u6027", "\\u8ca0\\u5411"], "type": "string"}, "\\u4fe1\\u4ef6\\u7a2e\\u985e": {"title": "\\u4fe1\\u4ef6\\u7a2e\\u985e", "description": "\\u5224\\u65b7\\u4fe1\\u4ef6\\u7684\\u7a2e\\u985e\\uff0c\\u4f8b\\u5982\\u91e3\\u9b5a\\u4fe1\\u6216\\u5b89\\u5168\\u4fe1\\u3002", "enum": ["\\u91e3\\u9b5a\\u4fe1", "\\u5b89\\u5168\\u4fe1", "\\u5176\\u4ed6

In [15]:
# Set up a parser + inject instructions into the prompt template.
email_parser = JsonOutputParser(pydantic_object=Email)

email_template = """<start_of_turn>user
你是一位網絡安全專家，你將會分析電子郵件的內容，請抓取出郵件中：
1.此郵件的情緒，正向或負向或中性
2.信件的種類，例如釣魚信或安全信
3.信件中提及的關鍵字，並以逗號','分隔

以下為一些範例：
```
範例1
郵件：我們發現您的賬戶存在可疑活動，請立即登錄確認。
抓取結果：{{
"情緒":"負向",
"信件種類":"釣魚信",
"關鍵字":"賬戶,可疑活動,登錄"
}}

範例2
郵件：本週五將舉行部門聚餐，請大家提前報名。
抓取結果：{{
"情緒":"正向",
"信件種類":"安全信",
"關鍵字":"聚餐,報名"
}}

範例3
郵件：您的包裹即將送達，請確認收貨地址。
抓取結果：{{
"情緒":"中性",
"信件種類":"安全信",
"關鍵字":"包裹,收貨地址"
}}

{format_instructions}
請使用繁體中文回答。
除了 JSON 內容外，不要回答任何其他內容

現在請幫我抓取以下句子的資訊：
句子：{query}
抓取結果：<end_of_turn>
<start_of_turn>model
"""

email_prompt = PromptTemplate(
    template=email_template,
    input_variables=["query"],
    partial_variables={"format_instructions": email_parser.get_format_instructions()},
)

# 假設 llm 是已經定義好的語言模型
chain = email_prompt | llm
chain.get_graph().print_ascii()

       +-------------+         
       | PromptInput |         
       +-------------+         
              *                
              *                
              *                
      +----------------+       
      | PromptTemplate |       
      +----------------+       
              *                
              *                
              *                
   +---------------------+     
   | HuggingFaceEndpoint |     
   +---------------------+     
              *                
              *                
              *                
+---------------------------+  
| HuggingFaceEndpointOutput |  
+---------------------------+  


手動建立 parse function

In [16]:
# 從 json 字串中抓取答案
def extract_answer_from_string(string_):
    string_ = string_.replace("\n", "")
    string_ = string_.replace("json", "")
    string_ = string_.replace("    ", "")
    string_ = string_.replace("```", "")
    try:
        # 可以成功轉換
        dict_result = json.loads(string_)
    except:
        print(string_)
        return string_
    return dict_result

# 從 dict 抓取情緒
def get_sentiment(row):
    sentiment = None
    if isinstance(row, dict):
        if ("情緒" in row.keys()):
            sentiment = row['情緒']
    return sentiment

將資料集中的重要資訊抓出

In [17]:
maildata['LLM'] = maildata.apply(lambda x: extract_answer_from_string(chain.invoke({"query": x['sentence']})), axis=1)
maildata

{"情緒": "正向","信件種類": "會議邀請","關鍵字": "hpsg,程序,國際,會議,語法,結構,語法結構,夏令,伊薩卡,紐約,七月,特展演講,標題,卡爾·波拉德,俄亥州州立大學,附屬詞,補語,案例指派,亞當·普雷皮奧爾科夫斯基,埃伯哈德·卡爾斯,圖 Bing 大大學,並列語義學,理由性無窮動詞語法,大衛·巴克斯特,伊利諾伊州大學 urbana-香馬爾尼,咖啡休息,分級分析,空缺依據句法結構的漢語,張永哲,伊利諾伊州大學 urbana-香馬爾尼,日本內部結構句法, chipped 小平,線性化與Wh萃取,塞爾沃-克羅地亞語的動詞語法,傑羅德·彭,埃伯哈德·卡爾斯,圖 Bing 大大學,午餐休息,塞爾沃-克羅地亞量化名詞補語,史蒂芬·威克斯勒,洛基薩·佐拉提克,德克薩斯州大學奧斯汀分校,安尼·阿貝爾,大學巴黎,丹尼爾·戈達爾,布reton的二級結構,布 Bob 伯斯利,威爾士大學班加羅,安德烈斯·高達爾,加州大學伯克利分校,西格芬·格里森,德馬特里·古利亞科普,尼德爾蘭格魯頓大學與新卡斯爾大學,法語主格結構,馬修·卡爾加諾,卡爾·波拉德,星期六，七月，特展演講，標題待定，彼得·斯里爾斯,德馬特里·古利亞科普,德語副詞短語，埃里哈德·希里希斯,埃伯哈德·卡爾斯,圖 Bing 大大學,津野奈華子,相依句的語義，提波·基斯,德馬特里·古利亞科普,咖啡休息，德語標記結構，安德烈斯·


,index,Email Text,Email Type,sentence,LLM
6974,7192,\nNow you can have HUNDREDS of lenders compete...,Phishing Email,Now you can have HUNDREDS of lenders compete f...,"{'情緒': '正向', '信件種類': '釣魚信', '關鍵字': '貸款,再入款,房屋貸..."
15453,15914,PUBLIC ANNOUNCEMENT:The new .NAME domain exten...,Phishing Email,PUBLIC ANNOUNCEMENTThe new NAME domain extensi...,"{'情緒': '正向', '信件種類': '新聞稿', '關鍵字': 'NAME域,個人域名..."
3275,3386,get the best price on your next car ! exclusiv...,Phishing Email,get the best price on your next car exclusive...,"{'情緒': '正向', '信件種類': '推廣信', '關鍵字': '車,優惠,報價,買車..."
4186,4333,"On Thu, 1 Aug 2002 17:10:48 +0100, John Hinsle...",Safe Email,On Thu Aug John Hinsley wrote No the prob...,"{'情緒': '正向', '信件種類': '安全信', '關鍵字': 'drivers,Me..."
6008,6198,santa barbara nexus ezine - great article pert...,Phishing Email,santa barbara nexus ezine great article perta...,"{'情緒': '正向', '信件種類': '安全信', '關鍵字': 'santa barb..."
...,...,...,...,...,...
3458,3574,"\n----- Original Message -----\nFrom: ""Jim Whi...",Safe Email,Original Message From Jim Whitehead For todd...,"{'情緒': '正向', '信件種類': '安全信', '關鍵字': ' toddler, ..."
11547,11901,germanic generative syntax newsletter germanic...,Safe Email,germanic generative syntax newsletter germanic...,"{'情緒': '正向', '信件種類': '安全信', '關鍵字': 'germanic g..."
5001,5164,a semiotic view of win and dos ? for your muse...,Safe Email,a semiotic view of win and dos for your musem...,"{'情緒': '正向', '信件種類': '評論性文章', '關鍵字': 'win,dos,..."
11284,11632,Have you thought of bumping up sylpheed-claws?...,Safe Email,Have you thought of bumping up sylpheedclaws ...,"{'情緒': '正向', '信件種類': '個人信', '關鍵字': 'sylpheedcl..."


In [18]:
maildata['sentiment'] = maildata.apply(lambda x: get_sentiment(x['LLM']), axis=1)
maildata.head()

,index,Email Text,Email Type,sentence,LLM,sentiment
6974,7192,\nNow you can have HUNDREDS of lenders compete...,Phishing Email,Now you can have HUNDREDS of lenders compete f...,"{'情緒': '正向', '信件種類': '釣魚信', '關鍵字': '貸款,再入款,房屋貸...",正向
15453,15914,PUBLIC ANNOUNCEMENT:The new .NAME domain exten...,Phishing Email,PUBLIC ANNOUNCEMENTThe new NAME domain extensi...,"{'情緒': '正向', '信件種類': '新聞稿', '關鍵字': 'NAME域,個人域名...",正向
3275,3386,get the best price on your next car ! exclusiv...,Phishing Email,get the best price on your next car exclusive...,"{'情緒': '正向', '信件種類': '推廣信', '關鍵字': '車,優惠,報價,買車...",正向
4186,4333,"On Thu, 1 Aug 2002 17:10:48 +0100, John Hinsle...",Safe Email,On Thu Aug John Hinsley wrote No the prob...,"{'情緒': '正向', '信件種類': '安全信', '關鍵字': 'drivers,Me...",正向
6008,6198,santa barbara nexus ezine - great article pert...,Phishing Email,santa barbara nexus ezine great article perta...,"{'情緒': '正向', '信件種類': '安全信', '關鍵字': 'santa barb...",正向


LLM抓出的結果表格顯示，有些抓出的信件種類與原始資料不太相同。LLM是利用關鍵字判別信件種類。

In [19]:
maildata['sentiment'].value_counts()

sentiment
正向    57
中性    22
負向    20
Name: count, dtype: int64